In [1]:
# Import dependences

import pandas as pd
import plotly.graph_objects as go

In [2]:
df = pd.read_csv("eb_2023.csv").set_index("Flow")
df = df.loc[:, 'Coal':].astype('float').reset_index()

In [3]:
fuels = ['Coal', 'Crude oil', 'Oil products', 'Natural gas', 'Nuclear', 'Hydro', 'Geo-therm., Solar etc.',
        'Biofuels and Waste', 'Electricity', 'Heat']
primary_flows = ['Production', 'Imports']
transformation_flows = ['Electricity plants', 'CHP plants', 'Heat plants',
                        'Blast furnaces', 'Gas works', 'Coke/pat. fuel/BKB/PB plants',
                        'Oil refineries', 'Petrochemical plants', 'Liquefaction plants',
                        'Other transformation', 
                        'Losses']
final_flows = ['INDUSTRY', 'TRANSPORT', 'OTHER', 'Energy industry own use','NON-ENERGY USE', 'Exports', 'Intl. marine bunkers', 
                 'Intl. aviation bunkers']

In [4]:
fuel_colors = {
    'Coal': '#4B4B4B',            # dark grey
    'Crude oil': '#A0522D',       # brown
    'Oil products': '#FFA500',    # orange
    'Natural gas': '#1E90FF',     # blue
    'Nuclear': '#9A32CD',         # purple
    'Hydro': '#00BFFF',           # light blue
    'Geo-therm., Solar etc.': '#228B22',  # green
    'Biofuels and Waste': '#FFD700',   # yellow
    'Electricity': '#E31A1C',     # red
    'Heat': '#FF6347'             # tomato
}

In [5]:
   
def hex_to_rgba(hex_color, alpha=0.6):
    hex_color = hex_color.lstrip('#')
    r, g, b = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
    return f"rgba({r},{g},{b},{alpha})"

def get_link_color(source, target):
    if source in fuel_colors:
        return hex_to_rgba(fuel_colors[source], 0.5)
    elif target in fuel_colors:
        return hex_to_rgba(fuel_colors[target], 0.5)
    else:
        return "rgba(180,180,180,0.3)"

In [6]:
x_pos = {}
for n in primary_flows:
    x_pos[n] = 0.05           # far left
for n in fuels:
    x_pos[n] = 0.30           # mid-left
for n in transformation_flows:
    x_pos[n] = 0.60           # mid-right
for n in final_flows:
    x_pos[n] = 0.85           # far right
    
import numpy as np

y_pos = {}
def distribute_y(group, start=0.05, end=0.95):
    spacing = np.linspace(start, end, len(group))
    for n, y in zip(group, spacing):
        y_pos[n] = y

In [7]:
links = []


# Primary flows
for flow in primary_flows:
    for fuel in fuels:
        val = df.loc[df['Flow'] == flow, fuel].values[0]
        if val > 0:
            links.append((flow, fuel, abs(val)))
        if val < 0:
            links.append((fuel, flow, abs(val)))

# 2. Transformation flows


for flow in transformation_flows:
    for fuel in fuels:
        val = df.loc[df['Flow'] == flow, fuel].values[0]
        if val > 0:
            links.append((flow, fuel, abs(val)))
        if val < 0:
            links.append((fuel, flow, abs(val)))
            

# 3. Final flows
for flow in final_flows:
    for fuel in fuels:
        val = df.loc[df['Flow'] == flow, fuel].values[0]
        if val > 0:
            links.append((fuel, flow, abs(val)))
        if val < 0:
            links.append((fuel, flow, abs(val)))


# --- Optional filtering to declutter ---

link_colors = [get_link_color(s, t) for s, t, _ in links]
# total = sum([v for _, _, v in links])
# links = [(s, t, v) for s, t, v in links if v > 0.005 * total]

# nodes = list(dict.fromkeys(primary_flows + fuels + transformation_flows + final_flows))
nodes = primary_flows + fuels + transformation_flows + final_flows
used_nodes = set([s for s, _, _ in links] + [t for _, t, _ in links])
nodes = [n for n in nodes if n in used_nodes]
node_indices = {node: i for i, node in enumerate(nodes)}

sources, targets, values = zip(*links)

node_colors = []
for n in nodes:
    if n in fuel_colors:
        node_colors.append(fuel_colors[n])
    elif n in primary_flows:
        node_colors.append("#CCCCCC")
    elif n in transformation_flows:
        node_colors.append("#87CEEB")
    elif n in final_flows:
        node_colors.append("#F4A460")
    else:
        node_colors.append("#D3D3D3")

distribute_y(primary_flows)
distribute_y(fuels)
distribute_y(transformation_flows)
distribute_y(final_flows)

fig = go.Figure(data=[go.Sankey(
    arrangement = 'snap',
    node=dict(
        pad=20,
        thickness=15,
        label=nodes,
        color= node_colors,
        x=[x_pos[n] for n in nodes],
        y=[y_pos[n] for n in nodes]
    ),
    link=dict(
        source=[node_indices[s] for s in sources],
        target=[node_indices[t] for t in targets],
        value=values,
        # color="rgba(0,150,250,0.4)"
        color=link_colors
    )
)])

# fig.update_layout(title_text="South Africa Energy Balance - 2023", font_size=10)
fig.update_layout(
    title_text="South Africa Energy Balance - 2023",
    font_size=11,
    annotations=[
        dict(
            text="<i>Source: Vuyo Mbam based on South Africa Energy Balance 2023 (IEA, DMRE)</i>",
            xref="paper", yref="paper",
            x=0, y=-0.12,          # position below chart
            showarrow=False,
            font=dict(size=10, color="gray"),
            xanchor='left',
        )
    ],
    margin=dict(t=80, b=80, l=10, r=10),
    width = 1400,
    height = 800
)
fig.show()
fig.write_html("SA_Energy_Flow.html")

In [8]:
primary_fuels = ['Coal', 'Biofuels and Waste', 'Crude oil', 'Natural gas', 'Nuclear', 'Geo-therm., Solar etc.','Hydro']
secondary_products = ['Electricity', 'Oil products',  'Heat']
primary_flows = ['Production', 'Imports']
transformation_flows = ['Electricity plants', 'Oil refineries', 'Liquefaction plants', 'CHP plants', 'Heat plants',
                        'Blast furnaces', 'Gas works', 'Coke/pat. fuel/BKB/PB plants', 'Petrochemical plants', 
                        'Other transformation']
final_flows = ['INDUSTRY', 'TRANSPORT', 'OTHER', 'Energy industry own use','NON-ENERGY USE', 'Exports', 'Intl. marine bunkers', 
                 'Intl. aviation bunkers', 
                        'Losses']


links = []



# Primary flows
for flow in primary_flows:
    for fuel in primary_fuels + secondary_products:
        val = df.loc[df['Flow'] == flow, fuel].values[0]
        if val > 0:
            links.append((flow, fuel, abs(val)))
        if val < 0:
            links.append((fuel, flow, abs(val)))

# 2. Transformation flows

for flow in transformation_flows:
    for fuel in primary_fuels:
        val = df.loc[df['Flow'] == flow, fuel].values[0]
        if val > 0:
            links.append((flow, fuel, abs(val)))
        if val < 0:
            links.append((fuel, flow, abs(val)))
            
for flow in transformation_flows:
    for fuel in secondary_products:
        val = df.loc[df['Flow'] == flow, fuel].values[0]
        if val > 0:
            links.append((flow, fuel, val))
        if val < 0:
            links.append((fuel, flow, abs(val)))
        

# 3. Final flows
for flow in final_flows:
    for fuel in primary_fuels + secondary_products:
        val = df.loc[df['Flow'] == flow, fuel].values[0]
        if val > 0:
            links.append((fuel, flow, abs(val)))
        if val < 0:
            links.append((fuel, flow, abs(val)))


# --- Optional filtering to declutter ---

link_colors = [get_link_color(s, t) for s, t, _ in links]
# total = sum([v for _, _, v in links])
# links = [(s, t, v) for s, t, v in links if v > 0.005 * total]

# nodes = list(dict.fromkeys(primary_flows + fuels + transformation_flows + final_flows))
nodes = primary_flows + primary_fuels + transformation_flows + secondary_products + final_flows
used_nodes = set([s for s, _, _ in links] + [t for _, t, _ in links])
nodes = [n for n in nodes if n in used_nodes]
node_indices = {node: i for i, node in enumerate(nodes)}

sources, targets, values = zip(*links)

node_colors = []
for n in nodes:
    if n in primary_fuels:
        node_colors.append(fuel_colors[n])
    elif n in primary_flows:
        node_colors.append("#CCCCCC")
    elif n in transformation_flows:
        node_colors.append("#87CEEB")
    elif n in secondary_products:
        node_colors.append("#FF0000")
    elif n in final_flows:
        node_colors.append("#F4A460")
    else:
        node_colors.append("#D3D3D3")

x_pos = {}
for n in primary_flows:
    x_pos[n] = 0.05           # far left
for n in fuels:
    x_pos[n] = 0.25           # mid-left
for n in transformation_flows:
    x_pos[n] = 0.55           # mid-right
for n in secondary_products:
    x_pos[n] = 0.75
for n in final_flows:
    x_pos[n] = 0.95  


distribute_y(primary_flows)
distribute_y(primary_fuels)
distribute_y(transformation_flows)
distribute_y(secondary_products)
distribute_y(final_flows)

fig = go.Figure(data=[go.Sankey(
    arrangement = 'snap',
    node=dict(
        pad=20,
        thickness=15,
        label=nodes,
        color= node_colors,
        x=[x_pos[n] for n in nodes],
        y=[y_pos[n] for n in nodes]
    ),
    link=dict(
        source=[node_indices[s] for s in sources],
        target=[node_indices[t] for t in targets],
        value=values,
        color="rgba(0,150,250,0.4)"
        # color=link_colors
    )
)])

# fig.update_layout(title_text="South Africa Energy Balance - 2023", font_size=10)
fig.update_layout(
    title_text="South Africa Energy Balance - 2023",
    font_size=11,
    annotations=[
        dict(
            text="<i>Source: Vuyo Mbam based on South Africa Energy Balance 2023 (IEA, DMRE)</i>",
            xref="paper", yref="paper",
            x=0, y=-0.12,          # position below chart
            showarrow=False,
            font=dict(size=10, color="gray"),
            xanchor='left',
        )
    ],
    margin=dict(t=80, b=80, l=10, r=10),
    width = 1400,
    height = 800
)
fig.show()
fig.write_html("SA_Energy_Flow.html")

In [9]:
df

,Flow,Coal,Crude oil,Oil products,Natural gas,Nuclear,Hydro,"Geo-therm., Solar etc.",Biofuels and Waste,Electricity,Heat,Total
0,Production,5.497286e+06,0.037647,3.122186e+05,655.000000,136647.87880,-29858.5516,78529.08028,652280.50,0.000000e+00,0.0,6.647759e+06
1,Imports,5.106914e+04,307174.689900,7.981012e+05,133923.421800,0.00000,0.0000,0.00000,0.00,2.960280e+04,0.0,1.319871e+06
2,Exports,-1.094444e+06,-1.734300,-1.361942e+05,-392.679000,0.00000,0.0000,0.00000,0.00,3.363480e+04,0.0,-1.197398e+06
3,Intl. marine bunkers,0.000000e+00,0.000000,-2.265659e+03,0.000000,0.00000,0.0000,0.00000,0.00,0.000000e+00,0.0,-2.265659e+03
4,Intl. aviation bunkers,0.000000e+00,0.000000,-2.421510e+03,0.000000,0.00000,0.0000,0.00000,0.00,0.000000e+00,0.0,-2.421510e+03
5,Stock changes,0.000000e+00,33857.766000,-1.150836e+04,0.000000,0.00000,0.0000,0.00000,0.00,0.000000e+00,0.0,2.234940e+04
6,TPES,4.453911e+06,341030.759200,9.579301e+05,134185.742800,136647.87880,-29858.5516,78529.08028,652280.50,6.323760e+04,0.0,6.787894e+06
7,Transfers,0.000000e+00,-180768.796500,0.000000e+00,0.000000,0.00000,0.0000,0.00000,0.00,0.000000e+00,0.0,-1.807688e+05
8,Statistical differences,-9.454954e+05,103172.115800,4.308569e+05,-388.390408,-29706.06061,-6671.1200,-16743.00506,0.00,4.755839e+05,0.0,1.060894e+04
9,Electricity plants,-2.375269e+06,0.000000,-3.044906e+04,0.000000,-106941.81820,36529.6716,-61786.07522,-4298.84,7.217971e+05,0.0,-1.820418e+06
